In [1]:
import os
import sys
if os.path.abspath('..') not in sys.path:
    sys.path.insert(0, os.path.abspath('..'))
import numpy as np
from IPython.display import display, Latex, Markdown
from lattices import Lattice, plotting, abaqus, elasticity_func

Create a BCC - SC lattice using fundamental representation

In [2]:
pos = np.array([
    [0,0,0],
    [0.5,0.5,0.5]
])
edges = np.array([
    [0,0],
    [0,0],
    [0,0],
    [1,0],
    [1,0],
    [1,0],
    [1,0],
    [1,0],
    [1,0],
    [1,0],
    [1,0],
])
tess_vec = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1],
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [1,1,0],
    [0,0,1],
    [1,0,1],
    [0,1,1],
    [1,1,1],
])

lat = Lattice(nodal_positions=pos, fundamental_edge_adjacency=edges, fundamental_tesselation_vecs=tess_vec)
lat = lat.create_windowed(min_edge_length=0.1, max_num_attempts=100)
plotting.plotly_unit_cell_3d(lat, show_uc_box=True).show()

We'll run simulations at two relative densities: 1% and 0.5%

Note that abaqus is required for this calculation. For interactive demo, we load the pre-computed results from file

In [3]:
rhos = [0.05, 0.1]
rs = [lat.calculate_edge_radius(r) for r in rhos]
lat.name = 'bcc-sc'
abaqus.write_abaqus_inp_normals(
    lat, 
    np.array(rs).reshape(1,-1), 
    metadata={
        'Relative densities': ','.join([str(rd) for rd in rhos]),
    }, fname='./bcc-sc/bcc-sc.inp'
)
ABQ_AVAILABLE = False
if ABQ_AVAILABLE:
    results = abaqus.run_abq_sim(['bcc-sc'], wdir='./bcc-sc')
else:
    results = [abaqus.get_results_from_json('./bcc-sc/bcc-sc.json')]

Calculate scaling law fit

In [4]:
S2 = np.stack([results[0]['compliance_tensors_M'][i] for i in rhos])
E_0_5 = 1/S2[0,0,0] # Young's modulus in x-direction at 0.5% relative density
E_1 = 1/S2[1,0,0] # ... at 1% relative density
fit = np.polyfit(np.log(rhos), np.log([E_0_5, E_1]), 1)
n = fit[0] # scaling exponent
A = np.exp(fit[1]) # proportionality constant
display(Markdown('Scaling law for the lattice is:'))
display(Markdown(r'$$ \frac{E}{E_s} = A \, \bar{\rho}^n $$'))
display(Markdown(r'$$ \frac{E}{E_s} = ' + f'{A:.2f}' + r'\, \bar{\rho}^{'+ f'{n:.2f}' + r'}$$'))

Scaling law for the lattice is:

$$ \frac{E}{E_s} = A \, \bar{\rho}^n $$

$$ \frac{E}{E_s} = 0.14\, \bar{\rho}^{1.02}$$

Plot the elasticity surface

In [5]:
plotting.plotly_elasticity_surf(elasticity_func.numpy_Mandel_to_cart_4(S2[0]), title='Compliance surface for BCC-SC lattice').show()